In [52]:
import pandas as pd
import datetime 
import numpy as np
import pickle
import matplotlib.pyplot as plt
import dtale
import re

%matplotlib inline

In [2]:
smileon = pd.read_csv('../data/smileon_edited.csv')

In [3]:
pd.set_option('display.max_columns', 118)
smileon.head()

,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,0,3638169,93115324,registration,9/30/2021 16:56,9/30/2021 16:57,1,123747,123747,9/30/2021 16:57,No,NaN,1960,Smyrna,Rutherford,TN,37167.0,1.0,NaN,English,African American,Female,$0 - $21 280,Other,SNAP; SSI/SSDI,NaN,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,Yes,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4431194,93115188,registration,9/30/2021 16:53,9/30/2021 16:53,0,123747,123747,9/30/2021 16:53,No,NaN,1959,Nashville,Davidson,TN,37211.0,1.0,NaN,English,Caucasian,Female,$0 - $21 280,Dental Clinic Staff,NaN,More than 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,NaN,Referral Follow-Up,Yes,Interfaith Dental Clinic - Nashville,Yes,NaN,No,Yes,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2941341,93115081,clinical,9/30/2021 16:50,9/30/2021 16:51,1,161979,161979,9/30/2021 16:51,No,NaN,1942,Memphis,Shelby,TN,38104.0,1.0,NaN,NaN,Asian,Male,$0 - $21 280,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,NaN,NaN,10/8/2019,Christ Community Health Services- Broad Avenue,Provider Site,No,Christ Community Health Services- Broad Avenue,Yes,NaN,NaN,No,Good,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/8/2019,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274 - Bitewings - four films - 5; 1110 - Proph...,No,Yes,No
3,3,3823933,93115032,clinical,9/30/2021 16:48,9/30/2021 16:49,1,161979,161979,9/30/2021 16:49,No,NaN,1960,Memphis,Shelby,TN,38109.0,1.0,NaN,NaN,African American,Male,$0 - $21 280,Dental Clinic Staff,NaN,More than 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,11/10/2020,Christ Community Health Services- Broad Avenue,

In [4]:
smileon = smileon.sort_values(['patient_id', 'entrydate'])

In [5]:
smileon = smileon.reset_index()
smileon.head(70)

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,13186,13186,-2,84708447,registration,2/19/2021 12:10,2/19/2021 12:56,46,119513,119513,2/19/2021 12:56,No,NaN,1954,Johnson City,Washington,TN,37604.0,4.0,NaN,Spanish,Hispanic,Female,$0 - $21 280,Dental Clinic Staff,NaN,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Northeast Tennessee Community Health Centers I...,Provider Site,Yes,Northeast Tennessee Community Health Centers I...,Yes,NaN,NaN,No,Dont know,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47543,47543,-2,62945363,clinical,5/15/2019 10:00,5/15/2019 11:00,60,119473,119473,5/15/2019 10:05,No,NaN,1953,NaN,NaN,NaN,NaN,2,NaN,NaN,Caucasian,Male,$32921-$37035,Dental Clinic Staff,NaN,Less than 12 months,No,No,NaN,No further questions from client,NaN,NaN,NaN,Healing Hands Health Center - Bristol,NaN,No,NaN,NaN,NaN,NaN,No,Dont know,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
2,42846,42846,-2,65438351,registration,8/13/2019 8:00,8/13/2019 8:05,5,133281,133281,8/13/2019 8:05,No,NaN,1957,Memphis,Shelby,TN,38127.0,1,NaN,English,African American,Female,$0 - $21 280,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Christ Community Health Services- Raleigh,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance; Fear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,228,228,-2,92968098,clinical,9/27/2021 12:56,9/27/2021 12:56,0,140162,140162,9/27/2021 12:56,No,NaN,1957,Mount Pleasant,Maury,TN,NaN,4.0,NaN,NaN,Caucasian,Female,$0 - $21 280,Dental Clinic Staff,SSI/SSDI,3 to 5 years,No,Yes,NaN,NaN,No,Yes,9/27/2021,TDH - Maury Co,Provider Site,Yes,NaN,NaN,NaN,NaN,Yes,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
#dropping the first 69 rows (rows 0-68) because they're weird - looks like test entries or something?  
#all data for patient's # -1 and -2
smileon = smileon.drop(range(0,69))
smileon = smileon.reset_index(drop=True)
smileon.head(10)

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,32182,32182,2384106,70281146,clinical,1/20/2020 13:25,1/20/2020 13:25,0,134549,134549,1/27/2020 13:25,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,Yes,Yes,Yes
1,53004,53004,2384106,59821075,clinical,1/9/2019 10:07,1/9/2019 10:07,0,118821,118821,1/28/2019 10:07,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,NaN,Yes,NaN
2,34169,34169,2384106,69642939,clinical,12/16/2019 10:35,12/16/2019 10:35,0,134549,134549,1/6/2020 10:35,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,No,Yes,No
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:59,10/9/2018 18:59,0,118821,118821,10/11/2018 18:59,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental

In [7]:
smileon['patient_id'].value_counts()

3212379    54
2441776    43
2758373    42
2431132    37
2416076    35
           ..
2628473     1
2530943     1
2511760     1
2635370     1
2823981     1
Name: patient_id, Length: 10709, dtype: int64

In [8]:
#NOTE :  THERE ARE 10709 UNIQUE PATIENTS IN THIS DF

In [9]:
smileon['reportversion'].value_counts()

clinical        40173
registration    17618
Name: reportversion, dtype: int64

In [10]:
#NOTE: BUT THERE ARE 17622 REGISTRATIONS.  MEANINS SOME PATIENTS, THE SAME PATIENT EVEN, REGISTERED MORE THAN ONCE

In [11]:
#I would really like to be able to see all the columns.  Going back to fix that
smileon.loc[smileon['patient_id']==2416076]

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
2477,32756,32756,2416076,70080607,clinical,1/13/2020 12:49,1/13/2020 12:52,3,117268,117268,1/20/2020 12:52,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nashville,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Poor,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272 - Bitewings - two films - 5; 1110 - Prophy...,No,Yes,No
2478,33854,33854,2416076,69737230,clinical,12/19/2019 11:37,12/19/2019 11:38,1,116749,116749,1/8/2020 11:38,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nashville,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Poor,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2330 - Resin one surface - Anterior - 15,No,Yes,Yes
2479,55777,55777,2416076,57225628,clinical,10/17/2018 10:15,10/17/2018 10:16,1,118821,118821,10/18/2018 10:16,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nashville,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Poor,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5221 - Partial denture - resin base - immediat...,NaN,NaN,NaN
2480,56291,56291,2416076,56828901,clinical,10/2/2018 13:47,10/2/2018 13:49,2,118821,118821,10/4/2018 13:49,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nas

In [12]:
#my df did not organize quite as well as i'd hoped - try again?
smileon = smileon.sort_values(['patient_id', 'callreportnum'])

#So, i was hoping that i'd see a row of needed procedures and then see them drop off over time to show progress ... but that's not what i have here.  What i have is procedures that were done.  And, i guess, how many times they were done.  Like oce_2330 - that's "resin one surface, anterior".  and sometimes it's in this patient's row x1, others x6.  so - i'm guessing that means they had six teeth done, with resin applied to one surface per tooth.

so, i can see that, with each visit, the patient had more work done.  and in the end, their plan was not completed and their full function was not restored.  Does that mean they weren't served ... ?
Obviously - no.  they were in better condition when they left than when they arrived - we see a ton of work done

Number of procedures per patient is a poor metric of success.  some need more help than others.  

perhaps a window could be - how many of these procedures are being offered overall.  a sum of each column.  finding the maximum column.  totalling some of them together since they're basically the same thing - veneers for example, irresespective of how many sides of the tooth.  that's important for billing - but not for treatment stats.

another window could be what complaint a person has when they arrive, and what treatments they received.  That will be harder to determine - but could be interesting.  Let's take a look at the complaints column.

In [13]:
smileon.head()

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
11,57818,57818,2384106,55083365,registration,8/1/2018 13:22,8/1/2018 14:09,47,118833,118833,7/31/2018 14:09,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,NaN,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,No,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,57743,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,57851,57851,2384106,55628771,clinical,8/1/2018 8:53,8/1/2018 8:54,1,118821,118821,8/21/2018 8:54,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7140 - Extraction - erupted tooth or exposed r...,NaN,NaN,NaN
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:5

#okay - they warned about that.  there's only 7 options for entry, but they're not broken up here.  There's a lot of possible combos.  

so, i need to break this up into 7 columns, and populate each with a 1 if that phrase exists in the 12months complaints column.  probably a regex function. they do at least show in order ... so, if a person is having problems biting foods, that will always be first in the list.

In [14]:
#let's see about some stats:
smileon.describe()

,index,Unnamed: 0,patient_id,callreportnum,calllength,callstaff_id,entrystaff_id,zip,s_num_lowerteeth,s_num_upperteeth,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2954,oce_2980,oce_3220,oce_4341,oce_6930,oce_6980,oce_7111,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951
count,57791.000000,57791.000000,5.779100e+04,5.779100e+04,57791.000000,57791.000000,57791.000000,54787.000000,6183.000000,5299.000000,1337.000000,12.000000,891.000000,50.000000,104.000000,46.000000,10.0,944.000000,940.000000,579.000000,428.000000,8.0,1226.000000,1341.000000,171.000000,2.0,134.000000,19.000000,74.000000,15.0,3.0,7.0,1396.000000,12.000000,2.0,9.000000,819.000000,5.000000,6.000000,19.000000,3.000000,486.000000,1.0,68.000000,225.000000,30.000000,80.000000,16.000000,1.0,4.0,12.0,2.0,1.0,26.000000,83.0
mean,28920.357997,28920.357997,3.083800e+06,7.417992e+07,4.097541,124761.271201,124851.273036,37551.853852,10.084910,9.824495,2.549738,2.416667,3.244669,1.280000,1.057692,1.043478,1.0,1.507415,1.361702,1.255613,1.343458,1.0,1.273246,1.158091,1.116959,1.0,1.037313,1.052632,1.108108,1.0,1.0,1.0,1.832378,1.083333,1.0,1.777778,1.800977,1.200000,1.166667,3.000000,3.666667,2.425926,1.0,1.073529,1.853333,1.233333,1.237500,2.187500,1.0,1.5,1.0,1.0,1.0,1.038462,1.0
std,16707.115214,16707.115214,5.808006e+05,1.086729e+07,10.497058,14704.883192,14248.340910,507.528006,3.183853,3.537183,2.173077,2.353270,3.039194,1.050559,0.234290,0.206185,0.0,1.027691,1.062990,0.628297,0.769001,0.0,0.579558,0.524394,0.403374,0.0,0.190240,0.229416,0.424186,0.0,0.0,0.0,0.814200,0.288675,0.0,1.394433,2.092763,0.447214,0.408248,4.558265,3.785939,2.378960,NaN,0.262944,0.931013,0.568321,0.483755,1.833712,NaN,1.0,0.0,0.0,NaN,0.196116,0.0
min,0.000000,0.000000,2.384106e+06,5.508336e+07,0.000000,-1.000000,60132.000000,24201.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
25%,14452.500000,14452.500000,2.633565e+06,6.540958e+07,1.000000,118845.000000,118845.000000,37146.000000,8.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.500000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
50%,28901.000000,28901.000000,2.891697e+06,7.157117e+07,1.000000,123747.000000,123747.000000,37334.000000,10.000000,10.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,2.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.0,1.000000,2.000000,1.000000,1.000000,1.500000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
75%,43409.500000,43409.500000,3.498203e+06,8.423406e+07,3.000000,134546.000000,134546.000000,38018.000000,12.000000,13.000000,3.000000,2.500000,4.000000,1.000000,1.000000,1.000000,1.0,2.000000,2.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,2.000000,1.000000,1.0,2.000000,2.000000,1.000000,1.000000,2.500000,5.000000,3.000000,1.0,1.000000,2.000000,1.000000,1.000000,3.000000,1.0,1.5,1.0,1.0,1.0,1.000000,1.0
max,57859.000000,57859.000000,4.581581e+06,9.350847e+07,735.000000,164262.000000,164262.000000,38654.000000,16.000000,16.000000,17.000000,8.000000,22.000000,8.000000,2.000000,2.000000,1.0,10.000000,27.000000,8.0000

oce_230:   x-rays          1337
oce_1351:  tooth sealant   12
oce_1354:  topical med     891
oce_2140:  filling 1s      50
oce_2150:  filling 2s      104
oce_2160:  filling 3s      46
oce_2161:  filling 4s      10
        TOTAL FILLINGS:        210
oce_2330:  resin 1s        944
oce_2331:  resin 2s        940
oce_2332:  resin 3s        579
oce_2335:  resin 4s        428
oce_2390:  crown           8
oce_2391:  postresin 1s    1226
oce_2392:  postresin 2s    1341
oce_2394:  postresin 3s    171
oce_2915:  prosthetic cast 2
oce_2920:  pros. crown     134
oce_2933:  steel crown     19
oce_2940:  sed. filling    74
oce_2954:  toothrebuild    15
oce_2980:  crown repair    3
oce_3220:  save rot.tooth  7
oce_4341:  plaque removal  1396
oce_6930:  recement bridge 12
oce_6980:  repair bridge   2
oce_7111:  extraction rem. 9
oce_7210:  surg. extract   819
oce_7220:  removal impacts 5
oce_7230:  removal impactpb 6
oce_7240:  removal impactcb 19
oce_7241:  removal complica 3
oce_7250:  surgery roots   486
oce_7261:  sinus closure   1
oce_7286:  biopsy          68
oce_7310:  alveoloplasty   225
oce_7311:  alveoloplasty   30
oce_7321:  alveoloplasty   80
oce_7410:  excision b.lesion 16
oce_7411:  excision b.lesion 1
oce_7471:  removal exostosis 4
oce_7510:  abscess drain   12
oce_7971:  pericoronal gin.  2
oce_7972:  surg. fibroustub. 1
oce_9110:  emergency treatm. 26
oce_9951:  occlusal adjust.  83

TOTAL cosmetic RESINS/RESTORATIONS:  5637
TOTAL SURGERIES:            1696

most common treatment: resin     2738
#2 most common:  plaque removal  1396
#3 most common:  xrays           1337

In [15]:
smileon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57791 entries, 11 to 57790
Columns: 118 entries, index to tp_clientrestored
dtypes: float64(48), int64(7), object(63)
memory usage: 52.5+ MB


In [16]:
smileon['s_problems12months'].value_counts()

Difficulty when biting or chewing foods                                                                                                                                                468
Experienced pain or discomfort                                                                                                                                                         381
Difficulty when biting or chewing foods; Experienced pain or discomfort                                                                                                                362
Dry mouth                                                                                                                                                                              344
Difficulty when biting or chewing foods; Difficulty with speech or pronouncing words; Dry mouth; Felt anxiety; Felt embarrassement; Avoided smiling; Experienced pain or discomfort    317
                                                                 

#okay - they warned about that. there's only 7 options for entry, but they're not broken up here. There's a lot of possible combos.

so, i need to break this up into 7 columns, and populate each with a 1 if that phrase exists in the 12months complaints column. probably a regex function. they do at least show in order ... so, if a person is having problems biting foods, that will always be first in the list.

In [17]:
'''s_difficulty_eating = []
s_difficulty_speaking = []
s_drymouth = []
s_felt_anxiety = []
s_felt_embarrassed = []
s_avoided_smiling = []
s_pain_discomfort = []'''

#looks like i didn't need to do this, so commenting out for now

's_difficulty_eating = []\ns_difficulty_speaking = []\ns_drymouth = []\ns_felt_anxiety = []\ns_felt_embarrassed = []\ns_avoided_smiling = []\ns_pain_discomfort = []'

In [18]:
#smileon[smileon['s_problems12months'].str.contains(r'(\w*\s+)chewing(\s+\w*)')]

In [19]:
#looks like i can't filter this column yet because of the nan values.  
#so - can i drop them, then maybe merge things back in?
smileon_drop = smileon.dropna(subset=['s_problems12months'])

In [20]:
smileon_drop['s_difficulty_eating'] = 0
smileon_drop['s_difficulty_speaking'] = 0
smileon_drop['s_drymouth'] = 0
smileon_drop['s_felt_anxiety'] = 0
smileon_drop['s_felt_embarrassed'] = 0
smileon_drop['s_avoided_smiling'] = 0
smileon_drop['s_pain_discomfort'] = 0

In [21]:
smileon_drop.head()

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,...,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
13,57743,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
19,57842,57842,2384744,55107015,clinical,8/1/2018 10:00,8/1/2018 10:22,22,118820,118820,8/1/2018 10:22,No,NaN,1954,Nashville,Davidson,TN,37211.0,1,NaN,NaN,Caucasian,Female,$21246-$24280,This encounter,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,No,Yes,8/1/2018,NaN,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Dont know,Referred to dental clinic,NaN,16.0,14.0,No,No,Difficulty when biting or chewing foods,twice daily,Yes,NaN,NaN,NaN,No,8/1/2018,Less than 12 months,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
34,57667,57667,2384833,55320630,clinical,8/9/2018 8:49,8/9/2018 8:50,1,118820,118820,8/9/2018 8:50,No,NaN,1942,Hartsville,Trousdale,TN,37074.0,1,NaN,NaN,Caucasian,Male,NaN,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/7/2018,NaN,NaN,No,NaN,Yes,NaN,NaN,No,Good,NaN,NaN,6.0,NaN,Upper,Upper,Difficulty when biting or chewing foods; Exper...,Once daily,Yes,No,NaN,Early care-within next several weeks,Yes,8/7/2018,Less than 12 months,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
39,57744,57744,2384922,55263902,clinical,8/7/2018 10:49,8/7/2018 10:50,1,116749,116749,8/7/2018 10:50,No,NaN,1937,Murfreesboro,Rutherford,TN,37129.0,1,NaN,NaN,NaN,Male,$0 - $21 280,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,No,Yes,8/1/2018,NaN,NaN,No,NaN,Yes,NaN,NaN,No,Good,

In [22]:
regex1 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)chewing(\s+\w*)')]  #3385
regex2 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)speech(\s+\w*)')]  #1348
regex3 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)Dry(\s+\w*)')]  #1454
regex4 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)anxiety(\S+\w*)')]  #1447
regex5 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)embarrassement(\S+\w*)')]  #1894
regex6 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)smiling(\S+\w*)')]  #1394
regex7 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)pain(\s+\w*)')]  #2835

In [23]:
#okay! that's working - now how to get these regex filters to populate the empty libaries with "1" or "0" values ... 
#and how to get those columns in the df?

In [24]:
'''for x in smileon_drop['s_problems12months']:
    if regex1:
        smileon_drop['s_difficulty_eating'] = 1'''
#of course that's too easy ... try again

'''for x in smileon_drop['s_problems12months']:
    if smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)chewing(\s+\w*)'):
        smileon_drop['s_difficulty_eating'] = 1'''
#still nope

"for x in smileon_drop['s_problems12months']:\n    if smileon_drop['s_problems12months'].str.contains(r'(\\w*\\s+)chewing(\\s+\\w*)'):\n        smileon_drop['s_difficulty_eating'] = 1"

In [25]:
smileon_drop['s_difficulty_eating'] = pd.np.where(smileon_drop['s_problems12months'].str.contains("chewing"),1,0)
#this code is working well!  looks like i don't even need regex at all ... 
smileon_drop['s_difficulty_speaking'] = pd.np.where(smileon_drop['s_problems12months'].str.contains("speech"),1,0)
smileon_drop['s_drymouth'] = pd.np.where(smileon_drop['s_problems12months'].str.contains("Dry"),1,0)
smileon_drop['s_felt_anxiety'] = pd.np.where(smileon_drop['s_problems12months'].str.contains("anxiety"),1,0)
smileon_drop['s_felt_embarrassed'] = pd.np.where(smileon_drop['s_problems12months'].str.contains("embarrassement"),1,0)
smileon_drop['s_avoided_smiling'] = pd.np.where(smileon_drop['s_problems12months'].str.contains("smiling"),1,0)
smileon_drop['s_pain_discomfort'] = pd.np.where(smileon_drop['s_problems12months'].str.contains("pain"),1,0)

In [26]:
smileon_drop.head(30)

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,...,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
13,57743,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,1,0,0,1
19,57842,57842,2384744,55107015,clinical,8/1/2018 10:00,8/1/2018 10:22,22,118820,118820,8/1/2018 10:22,No,NaN,1954,Nashville,Davidson,TN,37211.0,1,NaN,NaN,Caucasian,Female,$21246-$24280,This encounter,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,No,Yes,8/1/2018,NaN,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Dont know,Referred to dental clinic,NaN,16.0,14.0,No,No,Difficulty when biting or chewing foods,twice daily,Yes,NaN,NaN,NaN,No,8/1/2018,Less than 12 months,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0
34,57667,57667,2384833,55320630,clinical,8/9/2018 8:49,8/9/2018 8:50,1,118820,118820,8/9/2018 8:50,No,NaN,1942,Hartsville,Trousdale,TN,37074.0,1,NaN,NaN,Caucasian,Male,NaN,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/7/2018,NaN,NaN,No,NaN,Yes,NaN,NaN,No,Good,NaN,NaN,6.0,NaN,Upper,Upper,Difficulty when biting or chewing foods; Exper...,Once daily,Yes,No,NaN,Early care-within next several weeks,Yes,8/7/2018,Less than 12 months,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1
39,57744,57744,2384922,55263902,clinical,8/7/2018 10:49,8/7/2018 10:50,1,116749,116749,8/7/2018 10:50,No,NaN,1937,Murfreesboro,Rutherford,TN,37129.0,1,NaN,NaN,NaN,Male,$0 - $21 280,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,No,Yes,8/1/2018,NaN,NaN,No,NaN,Yes,NaN,NaN,No,Good,

In [27]:
smileon.head()

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
11,57818,57818,2384106,55083365,registration,8/1/2018 13:22,8/1/2018 14:09,47,118833,118833,7/31/2018 14:09,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,NaN,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,No,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,57743,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,57851,57851,2384106,55628771,clinical,8/1/2018 8:53,8/1/2018 8:54,1,118821,118821,8/21/2018 8:54,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7140 - Extraction - erupted tooth or exposed r...,NaN,NaN,NaN
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:5

In [28]:
#excellent!  now - let's try and merge drop back with the main df so i can compare treatments with complaints
#a left join/merge should do it
smileon_test2 = smileon.merge(smileon_drop, how = 'left', on = 'callreportnum')

In [29]:
smileon_test2.head()

,index_x,Unnamed: 0_x,patient_id_x,callreportnum,reportversion_x,callstart_x,callend_x,calllength_x,callstaff_id_x,entrystaff_id_x,entrydate_x,Reviewed_x,ReferralsMade_x,patient_bday_x,city_x,county_x,state_x,zip_x,demographic_household_x,demographic_lang_x,demographic_preferredlang_x,demographic_ethnic_x,demographic_gender_x,demographic_income_x,demographic_howyouhearprogram_x,demographic_otherassistance_x,demographic_lastdentistvisit_x,pe_insurance_x,pe_smileonprequalify_x,pe_nextstep_x,pe_nextstep2_x,ve_transitarranged_x,ve_eligibility_x,ve_verification_date_x,si_clinicname_x,si_typewhereclientenrolled_x,blq_troubleeating_x,blq_dentalclinic_x,blq_patientreferralcommitment_x,blq_did_needhelp_x,blq_do_needhelp_x,blq_qualityoflife_x,blq_mouthcondition_x,blq_nextstep_x,blq_nextstep2_x,s_num_lowerteeth_x,s_num_upperteeth_x,s_dentures_x,s_dentureswhileeating_x,s_problems12months_x,s_brushing_x,s_needforcare_x,s_rootfrags_x,s_lesions_x,s_urgency_x,s_decay_x,s_lastscreening_date_x,ci_lastseen_date_x,ci_apptmade_x,...,oce_1354_y,oce_2140_y,oce_2150_y,oce_2160_y,oce_2161_y,oce_2330_y,oce_2331_y,oce_2332_y,oce_2335_y,oce_2390_y,oce_2391_y,oce_2392_y,oce_2393_y,oce_2394_y,oce_2915_y,oce_2920_y,oce_2933_y,oce_2940_y,oce_2950_y,oce_2954_y,oce_2980_y,oce_3220_y,oce_4341_y,oce_4342_y,oce_6930_y,oce_6980_y,oce_7111_y,oce_7140_y,oce_7210_y,oce_7220_y,oce_7230_y,oce_7240_y,oce_7241_y,oce_7250_y,oce_7261_y,oce_7286_y,oce_7310_y,oce_7311_y,oce_7320_y,oce_7321_y,oce_7410_y,oce_7411_y,oce_7471_y,oce_7510_y,oce_7971_y,oce_7972_y,oce_9110_y,oce_9951_y,oce_WHATCAREGIVEN_y,tp_plancomplete_y,tp_plandeveloped_y,tp_clientrestored_y,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
0,57818,57818,2384106,55083365,registration,8/1/2018 13:22,8/1/2018 14:09,47,118833,118833,7/31/2018 14:09,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,NaN,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,No,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,57743,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,57851,57851,2384106,55628771,clinical,8/1/2018 8:53,8/1/2018 8:54,1,118821,118821,8/21/2018 8:54,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:59,10/9/2018 18:59,0,118821,118821,10/11/2018 18:59,No,NaN,1944,Murf

In [30]:
#ah!  no!  duplicates everywhere.  i have a list of all my columns.  annoying, but this should work:
smileon = smileon.merge(smileon_drop, how = 'left', on = [
    'patient_id', 'callreportnum', 'reportversion', 'callstart', 'callend', 'calllength', 'callstaff_id', 'entrystaff_id', 'entrydate', 'Reviewed', 'ReferralsMade', 'patient_bday', 'city', 'county', 'state', 'zip', 'demographic_household', 'demographic_lang', 'demographic_preferredlang', 'demographic_ethnic', 'demographic_gender', 'demographic_income', 'demographic_howyouhearprogram', 'demographic_otherassistance', 'demographic_lastdentistvisit', 'pe_insurance', 'pe_smileonprequalify', 'pe_nextstep', 'pe_nextstep2', 've_transitarranged', 've_eligibility', 've_verification_date', 'si_clinicname', 'si_typewhereclientenrolled', 'blq_troubleeating', 'blq_dentalclinic', 'blq_patientreferralcommitment', 'blq_did_needhelp', 'blq_do_needhelp', 'blq_qualityoflife', 'blq_mouthcondition', 'blq_nextstep', 'blq_nextstep2', 's_num_lowerteeth', 's_num_upperteeth', 's_dentures', 's_dentureswhileeating', 's_problems12months', 's_brushing', 's_needforcare', 's_rootfrags', 's_lesions', 's_urgency', 's_decay', 's_lastscreening_date', 'ci_lastseen_date', 'ci_apptmade', 'ci_transitprovided', 'cmi_clinic', 'cmi_transitproblem', 'cmi_findingcareproblem', 'cmi_clientoutcome', 'oce_230', 'oce_1351', 'oce_1354', 'oce_2140', 'oce_2150', 'oce_2160', 'oce_2161', 'oce_2330', 'oce_2331', 'oce_2332', 'oce_2335', 'oce_2390', 'oce_2391', 'oce_2392', 'oce_2393', 'oce_2394', 'oce_2915', 'oce_2920', 'oce_2933', 'oce_2940', 'oce_2950', 'oce_2954', 'oce_2980', 'oce_3220', 'oce_4341', 'oce_4342', 'oce_6930', 'oce_6980', 'oce_7111', 'oce_7140', 'oce_7210', 'oce_7220', 'oce_7230', 'oce_7240', 'oce_7241', 'oce_7250', 'oce_7261', 'oce_7286', 'oce_7310', 'oce_7311', 'oce_7320', 'oce_7321', 'oce_7410', 'oce_7411', 'oce_7471', 'oce_7510', 'oce_7971', 'oce_7972', 'oce_9110', 'oce_9951', 'oce_WHATCAREGIVEN', 'tp_plancomplete', 'tp_plandeveloped', 'tp_clientrestored'
])

In [31]:
smileon.head()

,index_x,Unnamed: 0_x,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,...,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored,index_y,Unnamed: 0_y,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
0,57818,57818,2384106,55083365,registration,8/1/2018 13:22,8/1/2018 14:09,47,118833,118833,7/31/2018 14:09,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,NaN,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,No,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,57743,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57743.0,57743.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,57851,57851,2384106,55628771,clinical,8/1/2018 8:53,8/1/2018 8:54,1,118821,118821,8/21/2018 8:54,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7140 - Extraction - erupted tooth or exposed r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:59,10/9/2018 18:59,0,118821,118821,10/11/2018 18:59,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,Na

In [32]:
smileon = smileon.drop(['Unnamed: 0_x'], axis =1)

In [33]:
#resorting back into order:
smileon = smileon.sort_values(['patient_id', 'callreportnum'])

In [34]:
smileon.head()

,index_x,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,...,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored,index_y,Unnamed: 0_y,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
0,57818,2384106,55083365,registration,8/1/2018 13:22,8/1/2018 14:09,47,118833,118833,7/31/2018 14:09,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,NaN,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,No,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57743.0,57743.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,57851,2384106,55628771,clinical,8/1/2018 8:53,8/1/2018 8:54,1,118821,118821,8/21/2018 8:54,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7140 - Extraction - erupted tooth or exposed r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56013,2384106,57055663,clinical,10/9/2018 18:59,10/9/2018 18:59,0,118821,118821,10/11/2018 18:59,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,

In [36]:
smileon.describe()

,index_x,patient_id,callreportnum,calllength,callstaff_id,entrystaff_id,zip,s_num_lowerteeth,s_num_upperteeth,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2954,oce_2980,oce_3220,oce_4341,oce_6930,oce_6980,oce_7111,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,index_y,Unnamed: 0_y,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
count,57791.000000,5.779100e+04,5.779100e+04,57791.000000,57791.000000,57791.000000,54787.000000,6183.000000,5299.000000,1337.000000,12.000000,891.000000,50.000000,104.000000,46.000000,10.0,944.000000,940.000000,579.000000,428.000000,8.0,1226.000000,1341.000000,171.000000,2.0,134.000000,19.000000,74.000000,15.0,3.0,7.0,1396.000000,12.000000,2.0,9.000000,819.000000,5.000000,6.000000,19.000000,3.000000,486.000000,1.0,68.000000,225.000000,30.000000,80.000000,16.000000,1.0,4.0,12.0,2.0,1.0,26.000000,83.0,5341.000000,5341.000000,5341.000000,5341.000000,5341.000000,5341.000000,5341.000000,5341.000000,5341.000000
mean,28920.357997,3.083800e+06,7.417992e+07,4.097541,124761.271201,124851.273036,37551.853852,10.084910,9.824495,2.549738,2.416667,3.244669,1.280000,1.057692,1.043478,1.0,1.507415,1.361702,1.255613,1.343458,1.0,1.273246,1.158091,1.116959,1.0,1.037313,1.052632,1.108108,1.0,1.0,1.0,1.832378,1.083333,1.0,1.777778,1.800977,1.200000,1.166667,3.000000,3.666667,2.425926,1.0,1.073529,1.853333,1.233333,1.237500,2.187500,1.0,1.5,1.0,1.0,1.0,1.038462,1.0,32647.139300,32647.139300,0.633776,0.252387,0.422393,0.311365,0.397678,0.436997,0.530799
std,16707.115214,5.808006e+05,1.086729e+07,10.497058,14704.883192,14248.340910,507.528006,3.183853,3.537183,2.173077,2.353270,3.039194,1.050559,0.234290,0.206185,0.0,1.027691,1.062990,0.628297,0.769001,0.0,0.579558,0.524394,0.403374,0.0,0.190240,0.229416,0.424186,0.0,0.0,0.0,0.814200,0.288675,0.0,1.394433,2.092763,0.447214,0.408248,4.558265,3.785939,2.378960,NaN,0.262944,0.931013,0.568321,0.483755,1.833712,NaN,1.0,0.0,0.0,NaN,0.196116,0.0,16837.052423,16837.052423,0.481817,0.434423,0.493987,0.463095,0.489464,0.496061,0.499097
min,0.000000,2.384106e+06,5.508336e+07,0.000000,-1.000000,60132.000000,24201.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,49.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14452.500000,2.633565e+06,6.540958e+07,1.000000,118845.000000,118845.000000,37146.000000,8.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.500000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,17986.000000,17986.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,28901.000000,2.891697e+06,7.157117e+07,1.000000,123747.000000,123747.000000,37334.000000,10.000000,10.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,2.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.0,1.000000,2.000000,1.000000,1.000000,1.500000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,35003.000000,35003.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,43409.500000,3.498203e+06,8.423406e+07,3.000000,134546.000000,1

In [38]:
visual_smileon = smileon[[
    'patient_id',
    'callreportnum',
    'reportversion',
    'oce_230', 
    'oce_1351','oce_1354', 
    'oce_2140','oce_2150','oce_2160','oce_2161','oce_2330','oce_2331','oce_2332','oce_2335','oce_2390','oce_2391','oce_2392', 
    'oce_2393','oce_2394','oce_2915','oce_2920','oce_2933','oce_2940','oce_2954','oce_2980',
    'oce_3220', 
    'oce_4341', 
    'oce_6930','oce_6980', 
    'oce_7111','oce_7210','oce_7220','oce_7230','oce_7240','oce_7241','oce_7250','oce_7261','oce_7286','oce_7310', 
    'oce_7311','oce_7321','oce_7410','oce_7411', 'oce_7471', 'oce_7510', 'oce_7971', 'oce_7972', 
    'oce_9110', 'oce_9951',  
    's_difficulty_eating','s_difficulty_speaking','s_drymouth','s_felt_anxiety','s_felt_embarrassed','s_avoided_smiling','s_pain_discomfort'
]]

In [56]:
visual_smileon = visual_smileon.groupby(visual_smileon.patient_id).sum()

In [57]:
visual_smileon.corr()

,callreportnum,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2954,oce_2980,oce_3220,oce_4341,oce_6930,oce_6980,oce_7111,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
callreportnum,1.000000,0.264210,0.036921,0.227210,0.033423,0.072139,0.053850,0.028860,0.277295,0.270557,0.240040,0.208023,0.051789,0.288217,0.314366,0.137490,0.010788,0.124143,0.013853,0.148718,0.079875,-0.004410,0.049454,0.288487,0.046590,-0.000152,0.001102,0.199706,0.012897,0.026768,0.015919,0.018122,0.170155,0.004413,0.136724,0.146114,0.052912,0.072898,0.033349,0.019203,0.044521,0.072411,0.014987,0.010374,0.068150,0.137053,0.420831,0.253239,0.317732,0.286755,0.347335,0.342791,0.362952
oce_230,0.264210,1.000000,0.006325,0.097570,0.002474,-0.002671,-0.003419,-0.002754,0.088798,0.089524,0.085997,0.062984,0.049750,0.061416,0.061513,0.061710,-0.003629,0.032045,-0.006429,0.051737,0.006308,0.028132,0.005399,0.062571,0.024388,-0.003629,0.004748,0.064858,-0.005443,0.002074,0.006143,0.014068,0.045998,0.021613,0.035985,0.059524,0.024392,0.025629,0.002739,-0.002566,-0.002117,0.006853,-0.003629,-0.002566,0.006007,0.046322,0.161433,0.089101,0.092971,0.095852,0.127105,0.119485,0.146873
oce_1351,0.036921,0.006325,1.000000,0.005614,-0.001178,0.004655,-0.001351,-0.000628,-0.001041,-0.001954,0.000063,-0.003360,-0.000616,0.006464,0.035472,0.006953,-0.000308,-0.001947,-0.000795,0.006089,-0.000792,-0.000377,-0.000576,0.037399,-0.000617,-0.000308,-0.000525,-0.000904,-0.000462,-0.000508,-0.000529,-0.000288,-0.000184,-0.000218,0.062529,-0.002755,-0.001049,-0.001692,-0.000671,-0.000218,-0.000377,-0.000698,-0.000308,-0.000218,-0.000994,0.062357,0.006767,-0.005674,0.007940,0.020485,-0.002996,-0.005433,0.020140
oce_1354,0.227210,0.097570,0.005614,1.000000,0.002093,0.004387,0.003377,0.009110,0.158949,0.091509,0.129916,0.121839,0.024209,0.105949,0.147714,0.159744,-0.001650,0.076925,-0.004262,0.122018,0.038749,-0.002021,-0.003087,0.106587,-0.001424,-0.001650,-0.002815,0.068712,-0.002475,-0.001282,0.000482,-0.001545,0.037952,-0.001167,0.046246,0.010814,0.039428,-0.007035,-0.003596,0.007476,-0.002021,0.011277,-0.001650,-0.001167,0.050966,0.084799,0.071624,0.014864,0.075094,0.045036,0.052947,0.040735,0.087458
oce_2140,0.033423,0.002474,-0.001178,0.002093,1.000000,0.051276,0.029697,0.047420,0.019902,0.008917,0.009832,0.008101,-0.001431,0.023782,0.008618,-0.000699,-0.000715,0.044517,-0.001848,0.004204,0.039236,-0.000876,-0.001339,0.017522,-0.001435,0.059133,-0.001220,-0.002695,-0.001073,-0.001180,0.002382,-0.000670,-0.007080,-0.000506,-0.003683,-0.006404,0.008591,0.002714,-0.001559,-0.000506,-0.000876,-0.001623,-0.000715,-0.000506,-0.002311,0.029602,0.018834,0.022425,0.052192,0.030963,0.023973,0.012153,0.022385
oce_2150,0.072139,-0.002671,0.004655,0.004387,0.051276,1.000000,0.218846,0.045211,0.053075,0.054557,0.021689,0.013506,-0.002398,0.061152,0.058843,0.029078,-0.001199,0.013667,0.072276,0.025383,-0.003085,-0.001468,-0.002243,0.028050,-0.002405,0.115506,-0.002045,0.002659,-0.001798,-0.001978,-0.002062,-0.001122,-0.008926,-0.000848,-0.006172,-0.008226,-0.004087,-0.000110,-0.002613,-0.000848,-0.001468,0.041408,-0.001199,-0.000848,0.024050,0.009451,0.026214,0.004736,0.049736,0.015871,0.014374,0.026929,0.032418
oce_2160,0.053850,-0.003419,-0.001351,0.003377,0.029697,0.218846,1.000000,0.073025,0.028699,0.013438,0.028214,0.015364,-0.001640,0.037068,0.030029,0.001086,0.090640,0.036450,0.068764,-0.004631,0.029277,-0.001004,-0.001534,0.006489,-0.001645,-0.000820,-0.001399,-0.005283,-0.001230,-0.001353,-0.001411,-0.000768,-0.008115,-0.000580,-0.004221,-0.007339,-0.002795,-0.004508,-0.001787,-0.000580,-0.001004,-0.001860,-0.000820,-0.00

So, there's no significant coorelation between complaints made and procedures received.  So ... no answers there.

The question: 
Explore the impact of the Smile On program and the outcomes that are produced for clients. Are clients seeing improvement through the Smile On program?

Is there any correlation between procedures given and improved outcomes?

Well, since we don't have any sort of "patient feedback upon completion of the program", nor do we even have records of many patients who even completed the program ... that's hard to answer with what we have.

But - I can catalogue and present what procedures were done based on their complaints. 

NOTE: THERE IS A STRONG CORRELATION BETWEEN :
1 - smiling and embarrassment
2 - pain and diffulty eating / anxiety and embarrassment
3 - smiling and difficulty eating
4 - embarrassment and diffuculty eating
5 - pain and smiling / pain and embarrassment


In [59]:
visual_smileon.head()

,callreportnum,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2954,oce_2980,oce_3220,oce_4341,oce_6930,oce_6980,oce_7111,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2384106,859604464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2384744,720961360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2384833,674745235,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2384897,55106455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2384922,110370524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [68]:
eating_visual_smileon = visual_smileon[visual_smileon['s_difficulty_eating']==1]
eating_visual_smileon.head()

,callreportnum,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2954,oce_2980,oce_3220,oce_4341,oce_6930,oce_6980,oce_7111,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2384106,859604464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2384744,720961360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2384833,674745235,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2384932,1371204478,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2384949,1328742759,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [67]:
eating_visual_smileon.sum(axis=0)

callreportnum            1.505516e+12
oce_230                  1.567000e+03
oce_1351                 1.100000e+01
oce_1354                 1.031000e+03
oce_2140                 1.500000e+01
oce_2150                 3.100000e+01
oce_2160                 1.100000e+01
oce_2161                 1.000000e+00
oce_2330                 5.260000e+02
oce_2331                 5.180000e+02
oce_2332                 2.870000e+02
oce_2335                 2.250000e+02
oce_2390                 3.000000e+00
oce_2391                 5.190000e+02
oce_2392                 5.730000e+02
oce_2394                 7.300000e+01
oce_2915                 0.000000e+00
oce_2920                 2.700000e+01
oce_2933                 5.000000e+00
oce_2940                 3.900000e+01
oce_2954                 5.000000e+00
oce_2980                 1.000000e+00
oce_3220                 2.000000e+00
oce_4341                 8.670000e+02
oce_6930                 9.000000e+00
oce_6980                 1.000000e+00
oce_7111    

okay!  that worked- but I have so many different columns ... if i'm going to visualize this, i need to compress it a bit.  a lot of these are the same procedures done with multiple variations.  bet i can combine that with integrity

In [71]:
visual_smileon = visual_smileon.rename(columns={
    'oce_230':'imaging',
    'oce_1351':'sealant',
    'oce_1354':'medicament',
    'oce_2140':'amalgam_cavityfilling1',
    'oce_2150':'amalgam_cavityfilling2',
    'oce_2160':'amalgam_cavityfilling3',
    'oce_2161':'amalgam_cavityfilling4',
    'oce_2330':'anterior_dentalresin1',
    'oce_2331':'anterior_dentalresin2',
    'oce_2332':'anterior_dentalresin3',
    'oce_2335':'anterior_dentalresin4',
    'oce_2390':'resincrown',
    'oce_2391':'posterior_dentalresin1',
    'oce_2392':'posterior_dentalresin2',
    'oce_2394':'posterior_dentalresin4',
    'oce_2915':'recement_prosthetic',
    'oce_2920':'recement_crown',
    'oce_2933':'prefab_steel_crown',
    'oce_2940':'sedative_filling',
    'oce_2954':'prefab_prosthetic',
    'oce_2980':'crown_repair',
    'oce_3220':'therapeutic_pulpotomy',
    'oce_4341':'plaque_removal',
    'oce_6930':'recement_bridge',
    'oce_6980':'bridge_repair',
    'oce_7111':'toothremnant_extraction',
    'oce_7210':'surgical_extraction',
    'oce_7220':'impactedtooth_removal1',
    'oce_7230':'impactedtooth_removal2',
    'oce_7240':'impactedtooth_removal3',
    'oce_7241':'impactedtooth_removal4',
    'oce_7250':'surgery_rootremoval',
    'oce_7261':'sinus_perforation_closure',
    'oce_7286':'biopsy',
    'oce_7310':'alveoloplasty1',
    'oce_7311':'alveoloplasty2',
    'oce_7321':'alveoloplasty3',
    'oce_7410':'lesion_excision1',
    'oce_7411':'lesion_excision2',
    'oce_7471':'bonelesion_excision1',
    'oce_7510':'abscess_drain',
    'oce_7971':'pericoronalgingiva_excision',
    'oce_7972':'surgery_fibroustuberosity_reduction',
    'oce_9110':'emergencytreatment_dentalpain',
    'oce_9951':'occlusal_adjustment'
})

In [80]:
visual_smileon.head(20)

,callreportnum,imaging,sealant,medicament,amalgam_cavityfilling1,amalgam_cavityfilling2,amalgam_cavityfilling3,amalgam_cavityfilling4,anterior_dentalresin1,anterior_dentalresin2,anterior_dentalresin3,anterior_dentalresin4,resincrown,posterior_dentalresin1,posterior_dentalresin2,posterior_dentalresin4,recement_prosthetic,recement_crown,prefab_steel_crown,sedative_filling,prefab_prosthetic,crown_repair,therapeutic_pulpotomy,plaque_removal,recement_bridge,bridge_repair,toothremnant_extraction,surgical_extraction,impactedtooth_removal1,impactedtooth_removal2,impactedtooth_removal3,impactedtooth_removal4,surgery_rootremoval,sinus_perforation_closure,biopsy,alveoloplasty1,alveoloplasty2,alveoloplasty3,lesion_excision1,lesion_excision2,bonelesion_excision1,abscess_drain,pericoronalgingiva_excision,surgery_fibroustuberosity_reduction,emergencytreatment_dentalpain,occlusal_adjustment,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort,amalgam_cavityfilling
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2384106,859604464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2384744,720961360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2384833,674745235,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
2384897,55106455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2384922,110370524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2384932,1371204478,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
2384949,1328742759,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2384963,533321736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2384971,627279669,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0


In [88]:
visual_smileon['amalgam_cavityfilling']=visual_smileon.iloc[:, -50:-46].sum(axis=1)
#note- had to adjust my iloc because first i had the positions wrong, 
#then again because of the new column added to the end throwing the position #s off - running over would produce wrong #s

#visual_smileon['amalgam_cavityfiling']=visual_smileon.iloc[:, -49:-45].sum(axis=1)

In [112]:
testing = visual_smileon[visual_smileon['alveoloplasty3']>=1]
testing.head()

,callreportnum,imaging,sealant,medicament,amalgam_cavityfilling1,amalgam_cavityfilling2,amalgam_cavityfilling3,amalgam_cavityfilling4,anterior_dentalresin1,anterior_dentalresin2,anterior_dentalresin3,anterior_dentalresin4,resincrown,posterior_dentalresin1,posterior_dentalresin2,posterior_dentalresin4,recement_prosthetic,recement_crown,prefab_steel_crown,sedative_filling,prefab_prosthetic,crown_repair,therapeutic_pulpotomy,plaque_removal,recement_bridge,bridge_repair,toothremnant_extraction,surgical_extraction,impactedtooth_removal1,impactedtooth_removal2,impactedtooth_removal3,impactedtooth_removal4,surgery_rootremoval,sinus_perforation_closure,biopsy,alveoloplasty1,alveoloplasty2,alveoloplasty3,lesion_excision1,lesion_excision2,bonelesion_excision1,abscess_drain,pericoronalgingiva_excision,surgery_fibroustuberosity_reduction,emergencytreatment_dentalpain,occlusal_adjustment,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort,amalgam_cavityfilling,dental_resin,impacted_tooth_removal,alveoloplasty,lesion_excision
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2416081,777886206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2445886,490764032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2464205,421209679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466994,1856070993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2467467,230572576,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


In [98]:
visual_smileon['dental_resin']=visual_smileon.iloc[:, -47:-39].sum(axis=1)
#visual_smileon['dental_resin']=visual_smileon.iloc[:, -46:-38].sum(axis=1)

In [102]:
visual_smileon['impacted_tooth_removal']=visual_smileon.iloc[:, -27:-24].sum(axis=1)

In [106]:
visual_smileon['alveoloplasty']=visual_smileon.iloc[:, -21:-19].sum(axis=1)

In [110]:
visual_smileon['lesion_excision']=visual_smileon.iloc[:, -19:-18].sum(axis=1)

In [113]:
visual_smileon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10709 entries, 2384106 to 4581581
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   callreportnum                        10709 non-null  int64  
 1   imaging                              10709 non-null  float64
 2   sealant                              10709 non-null  float64
 3   medicament                           10709 non-null  float64
 4   amalgam_cavityfilling1               10709 non-null  float64
 5   amalgam_cavityfilling2               10709 non-null  float64
 6   amalgam_cavityfilling3               10709 non-null  float64
 7   amalgam_cavityfilling4               10709 non-null  float64
 8   anterior_dentalresin1                10709 non-null  float64
 9   anterior_dentalresin2                10709 non-null  float64
 10  anterior_dentalresin3                10709 non-null  float64
 11  anterior_dentalresin

In [115]:
visual_smileon2 = visual_smileon[[
    'callreportnum',
    'imaging',
    'plaque_removal',
    'sealant',
    'medicament',
    'amalgam_cavityfilling',
    'dental_resin',
    'recement_prosthetic',    'recement_crown',    'recement_bridge','crown_repair','bridge_repair',
    'prefab_steel_crown',
    'sedative_filling',
    'prefab_prosthetic',
    'therapeutic_pulpotomy',
    'toothremnant_extraction',    'surgical_extraction',    'impacted_tooth_removal',    'surgery_rootremoval',
    'sinus_perforation_closure',
    'biopsy',
    'alveoloplasty',
    'lesion_excision',
    'bonelesion_excision1',
    'abscess_drain',
    'pericoronalgingiva_excision',
    'surgery_fibroustuberosity_reduction',
    'emergencytreatment_dentalpain',
    'occlusal_adjustment',
    's_difficulty_eating','s_difficulty_speaking','s_drymouth','s_felt_anxiety','s_felt_embarrassed','s_avoided_smiling','s_pain_discomfort'
]]  

In [116]:
visual_smileon2.head()

,callreportnum,imaging,plaque_removal,sealant,medicament,amalgam_cavityfilling,dental_resin,recement_prosthetic,recement_crown,recement_bridge,crown_repair,bridge_repair,prefab_steel_crown,sedative_filling,prefab_prosthetic,therapeutic_pulpotomy,toothremnant_extraction,surgical_extraction,impacted_tooth_removal,surgery_rootremoval,sinus_perforation_closure,biopsy,alveoloplasty,lesion_excision,bonelesion_excision1,abscess_drain,pericoronalgingiva_excision,surgery_fibroustuberosity_reduction,emergencytreatment_dentalpain,occlusal_adjustment,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2384106,859604464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2384744,720961360,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2384833,674745235,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2384897,55106455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2384922,110370524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [121]:
#combining more columns that make sense - to shrink the diameters of my df
visual_smileon2['dentalwork_repairs']=visual_smileon2.iloc[:, -31:-27].sum(axis=1)

In [138]:
testing2 = visual_smileon2[visual_smileon2['alveoloplasty']>=1]
testing2.head()

,callreportnum,imaging,plaque_removal,sealant,medicament,amalgam_cavityfilling,dental_resin,recement_prosthetic,recement_crown,recement_bridge,crown_repair,bridge_repair,prefab_steel_crown,sedative_filling,prefab_prosthetic,therapeutic_pulpotomy,toothremnant_extraction,surgical_extraction,impacted_tooth_removal,surgery_rootremoval,sinus_perforation_closure,biopsy,alveoloplasty,lesion_excision,bonelesion_excision1,abscess_drain,pericoronalgingiva_excision,surgery_fibroustuberosity_reduction,emergencytreatment_dentalpain,occlusal_adjustment,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort,dentalwork_repairs,tooth_extractions_removals,excision_lesions
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2384106,859604464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,12.0,0.0
2384932,1371204478,3.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2385046,650311958,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0
2385168,504248811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,15.0,0.0
2386075,587503301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [123]:
visual_smileon2['tooth_extractions_removals']=visual_smileon2.iloc[:, -22:-19].sum(axis=1)

In [134]:
visual_smileon2['excision_lesions']=visual_smileon2.iloc[:, -16:-15].sum(axis=1)

In [141]:
visual_smileon2 = visual_smileon2[[
    's_difficulty_eating','s_difficulty_speaking','s_drymouth','s_felt_anxiety','s_felt_embarrassed','s_avoided_smiling','s_pain_discomfort',
    'imaging',
    'plaque_removal',
    'sealant',
    'medicament',
    'amalgam_cavityfilling',
    'dental_resin',
    'dentalwork_repairs',
    'prefab_steel_crown',
    'sedative_filling',
    'prefab_prosthetic',
    'therapeutic_pulpotomy',
    'tooth_extractions_removals',
    'sinus_perforation_closure',
    'biopsy',
    'alveoloplasty',
    'excision_lesions',
    'abscess_drain',
    'pericoronalgingiva_excision',
    'surgery_fibroustuberosity_reduction',
    'emergencytreatment_dentalpain',
    'occlusal_adjustment'
]]  

In [142]:
visual_smileon2.head()

,s_difficulty_eating,s_difficulty_speaking,s_drymouth,s_felt_anxiety,s_felt_embarrassed,s_avoided_smiling,s_pain_discomfort,imaging,plaque_removal,sealant,medicament,amalgam_cavityfilling,dental_resin,dentalwork_repairs,prefab_steel_crown,sedative_filling,prefab_prosthetic,therapeutic_pulpotomy,tooth_extractions_removals,sinus_perforation_closure,biopsy,alveoloplasty,excision_lesions,abscess_drain,pericoronalgingiva_excision,surgery_fibroustuberosity_reduction,emergencytreatment_dentalpain,occlusal_adjustment
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2384106,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2384744,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2384833,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2384897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2384922,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
visual_smileon2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10709 entries, 2384106 to 4581581
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   s_difficulty_eating                  10709 non-null  float64
 1   s_difficulty_speaking                10709 non-null  float64
 2   s_drymouth                           10709 non-null  float64
 3   s_felt_anxiety                       10709 non-null  float64
 4   s_felt_embarrassed                   10709 non-null  float64
 5   s_avoided_smiling                    10709 non-null  float64
 6   s_pain_discomfort                    10709 non-null  float64
 7   imaging                              10709 non-null  float64
 8   plaque_removal                       10709 non-null  float64
 9   sealant                              10709 non-null  float64
 10  medicament                           10709 non-null  float64
 11  amalgam_cavityfillin

In [143]:
visual_smileon2.to_csv('../data/smileon_visualready.csv')

In [58]:
#let's play with this dtale tool that robert found!  
#d=dtale.show(visual_smileon)
#d.open_browser()

Executing shutdown due to inactivity...


2021-11-16 22:14:49,163 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2021-11-16 22:14:57,344 - INFO     - Executing shutdown...
